In [7]:
import timeit
start = timeit.default_timer()

import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'
import sys
print(sys.version)

3.10.9 (main, Mar  1 2023, 18:23:06) [GCC 11.2.0]


In [8]:
import pyarrow as pa
import geohash_tools as gh
import pyspark as ps

In [9]:
sp = spark.read.load('hdfs://orion11:14001/nam/2019/01/05/*',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

schema = sp.schema

sp = spark.read.schema(schema).load('hdfs://orion11:14001/nam/201*/*/*/*',
                     format='csv',
                     sep='\t',
                     header='true')

In [10]:
sp = sp.select(['2_lat','3_lon','snow_depth_surface'])

In [11]:
rddf = sp.rdd \
    .map(lambda x: ( gh.encode(x[0],x[1], 6), (x[2], 1.0)) ) \
    .reduceByKey(lambda av,bv: (av[0]+bv[0], av[1]+bv[1]) ) \
    .map(lambda x: (x[0], x[1][0]/x[1][1])) \
    .sortBy(lambda x: x[1], ascending=False) \
    .cache()

In [12]:
print(rddf.count())
strangely_snowy = rddf.filter(lambda x: x[1] > 1.4).collect()
print(len(strangely_snowy))

523165
40


In [13]:
print(strangely_snowy)
#   2k
#  60k
# 262k

[('c41ueb', 2.7801569015387004), ('c41uhb', 2.778778995762526), ('c41uxk', 2.5776117425280503), ('c44hg1', 2.5740536200731756), ('c41v48', 2.447659330213517), ('c41vnk', 2.3215498501809746), ('c44j61', 2.3180099222026356), ('c44jc1', 2.0594607913434295), ('c41vtk', 2.059222234628628), ('c41v98', 2.010015243177979), ('c41vbx', 1.9923131194235015), ('c41yhk', 1.8697448553075093), ('c41wrx', 1.82848115190426), ('f2w7rv', 1.7930811062931407), ('c41z4k', 1.787254150867617), ('c41yek', 1.7856631037196025), ('c44h0k', 1.7791885073649456), ('c44hk1', 1.7766683632494222), ('c445v1', 1.7733938505418403), ('c41gtb', 1.7698529617324552), ('c4459s', 1.769420615270361), ('c41z9h', 1.6674425141093867), ('c432rd', 1.6615142480444045), ('c4387r', 1.6598645730263537), ('c41xbf', 1.6546297481815886), ('c41xur', 1.652405921322383), ('c1nmmz', 1.6218227029455232), ('c1nkyt', 1.6203928114292783), ('c1nuq5', 1.5678783532148015), ('c1nhxt', 1.5529354519490612), ('c1nkg1', 1.5521080150537543), ('c43b05', 1.535

In [14]:
for i in range(len(strangely_snowy)):
    closest = rddf \
        .map(lambda x: (gh.distance(x[0], strangely_snowy[i][0]), x[1])) \
        .sortByKey() \
        .take(4)[1:] \

    strangely_snowy[i] = strangely_snowy[i] + \
    (sum([c[0] for c in closest]) / len(closest), sum([c[1] for c in closest]) / len(closest))

for e in strangely_snowy:
    print('Location {} has a snowy index of {:.2f}'.format(e[0], e[1]))
    print('Its closest neighbors (average distance {}km), average a snowy index of {:.2f}'.format(int(e[2]), e[3]))
    print('Is that strangely snowy or what?\n')

Location c41ueb has a snowy index of 2.78
Its closest neighbors (average distance 44461km), average a snowy index of 0.00
Is that strangely snowy or what?

Location c41uhb has a snowy index of 2.78
Its closest neighbors (average distance 18246km), average a snowy index of 0.00
Is that strangely snowy or what?

Location c41uxk has a snowy index of 2.58
Its closest neighbors (average distance 20679km), average a snowy index of 0.02
Is that strangely snowy or what?

Location c44hg1 has a snowy index of 2.57
Its closest neighbors (average distance 33823km), average a snowy index of 0.00
Is that strangely snowy or what?

Location c41v48 has a snowy index of 2.45
Its closest neighbors (average distance 10874km), average a snowy index of 0.04
Is that strangely snowy or what?

Location c41vnk has a snowy index of 2.32
Its closest neighbors (average distance 14780km), average a snowy index of 0.03
Is that strangely snowy or what?

Location c44j61 has a snowy index of 2.32
Its closest neighbors 

In [15]:
print(timeit.default_timer()-start)

885.9681626110105
